In this notebook we make simple synthetic dataset to test our anonimization model.

In [ ]:
from datasets import load_dataset
import numpy as np
import pandas as pd 

dataset = load_dataset("parquet", data_files="/kaggle/input/5k-supreme/2024-5K-supreme-court-decisions-deduplicated.parquet")
dataset = dataset['train']

print(dataset)

columns_to_keep = ["id","text", "person_count", "person_occurrences"]
dataset = dataset.select_columns(columns_to_keep)

print(dataset)


Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['id', 'text', 'number_count', 'information_count', 'person_count', 'address_count', 'sum_of_unique_entities', 'number_occurrences', 'information_occurrences', 'person_occurrences', 'address_occurrences', '__index_level_0__'],
    num_rows: 5125
})
Dataset({
    features: ['id', 'text', 'person_count', 'person_occurrences'],
    num_rows: 5125
})


In [3]:
DEFAULT_BASE_URL = "https://bedrock-runtime.us-west-2.amazonaws.com/openai/v1"
DEFAULT_MODEL = "openai.gpt-oss-120b-1:0"

In [ ]:
from openai import OpenAI
import re
import json

client = OpenAI(
    base_url=DEFAULT_BASE_URL,
    api_key="ABSKQmVkcm9ja0FQSUtleS1maGwyLWF0LTA2OTM3MjkxNDI4ODpyV1RraG5peHVUREhQZkEwUEkra2hDS3hXZ2dJb05mVzZVSjRmakF2WmNzeGdhdDBSUVEzc0EwYVNXUT0="
)


def llm_generate_person_entities(text, entities):
    prompt = f"""
You are given a Ukrainian legal text and a list of entities marking anonymized persons (like ОСОБА_1, ОСОБА_2, etc.).
Each entity has start, end, and text fields.

TASK:
- Replace each anonymized label with a **realistic but uncommon Ukrainian full name**.
- The same label must always refer to the same person.
- Person name format should be ПІБ - Прізвише Імя Побатькові
- Inflect names naturally according to Ukrainian grammar (case, gender, etc.) depending on context.
- **DO NOT** add any explanations, reasoning, or extra text.
- Return ONLY valid JSON inside <json>...</json> tags in this format:
  <json>[{{"start": int, "end": int, "text": "New Name"}}, ...]</json>

Text:
{text}

Entities:
{entities}
"""

    response = client.chat.completions.create(
        model=DEFAULT_MODEL,
        messages=[{"role": "user", "content": prompt}],
        temperature=0.75
    )

    output_text = response.choices[0].message.content
    output_text = re.sub(r"<reasoning>.*?</reasoning>", "", output_text, flags=re.S | re.I)

    matches = re.findall(r"<json>(.*?)</json>", output_text, flags=re.S | re.I)
    if not matches:
        print("No <json>...</json> found in output.")
        return None

    json_text = matches[-1].strip()

    try:
        return json.loads(json_text)
    except json.JSONDecodeError:
        print("Invalid JSON:\n", json_text)
        return None

In [31]:
row = dataset[0]
text = row["text"]
entities = row["person_occurrences"]


In [41]:
result = llm_generate_person_entities(text, entities)

print("\n--- Result ---")
print(json.dumps(result, indent=2, ensure_ascii=False))


--- Result ---
[
  {
    "start": 504,
    "end": 511,
    "text": "Ковалевського Олександра Вікторовича"
  },
  {
    "start": 765,
    "end": 772,
    "text": "Ковалевського Олександра Вікторовича"
  },
  {
    "start": 1332,
    "end": 1339,
    "text": "Ковалевський Олександр Вікторович"
  },
  {
    "start": 1550,
    "end": 1557,
    "text": "Ковалевського Олександра Вікторовича"
  },
  {
    "start": 1841,
    "end": 1848,
    "text": "Ковалевського Олександра Вікторовича"
  },
  {
    "start": 609,
    "end": 616,
    "text": "Бурса Олена Михайлівна"
  },
  {
    "start": 929,
    "end": 936,
    "text": "Бурса Олена Михайлівна"
  },
  {
    "start": 949,
    "end": 956,
    "text": "Бурса Олена Михайлівна"
  }
]


In [ ]:
results = []

for i in range(min(50, len(dataset))):
    row = dataset[i]
    text = row["text"]
    entities = row["person_occurrences"]
    
    new_entities = llm_generate_person_entities(text, entities)
    results.append({
        "id": row["id"],
        "text": text,
        "generated_entities": new_entities
    })
    print(f"Processed row {i+1} / {min(50, len(dataset))}")

with open("first_50_rows_anonymized.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print("Done: first 50 rows processed and saved to first_50_rows_anonymized.json")

Processed row 1 / 50
Processed row 2 / 50
Processed row 3 / 50
Processed row 4 / 50
Processed row 5 / 50
Processed row 6 / 50
Processed row 7 / 50
Processed row 8 / 50
Processed row 9 / 50
Processed row 10 / 50
Processed row 11 / 50
Processed row 12 / 50
Processed row 13 / 50
Processed row 14 / 50
Processed row 15 / 50
Processed row 16 / 50
Processed row 17 / 50
Processed row 18 / 50
Processed row 19 / 50
Processed row 20 / 50
Processed row 21 / 50
Processed row 22 / 50
Processed row 23 / 50
Processed row 24 / 50
Processed row 25 / 50
Processed row 26 / 50
Processed row 27 / 50
Processed row 28 / 50
Processed row 29 / 50
Processed row 30 / 50
Processed row 31 / 50
Processed row 32 / 50
Processed row 33 / 50
Processed row 34 / 50
Processed row 35 / 50
Processed row 36 / 50
Processed row 37 / 50
Processed row 38 / 50
Processed row 39 / 50
Processed row 40 / 50
Processed row 41 / 50
Processed row 42 / 50
Processed row 43 / 50
Processed row 44 / 50
Processed row 45 / 50
Processed row 46 / 